In [1]:
import os
import pandas as pd
import numpy as np
import sympy as sym
from sympy import coth
from sympy import sqrt

In [2]:
def coth1(x):
    if type(x) == sym.core.numbers.Float:
        x = coth(x)
    else:
        x = np.cosh(x)/np.sinh(x)
    return(x)

def ss_extension(F,kbp_ss =20,kT = 4.11,Lp_s = 0.6,Lc_s_lambda = 27000,S_s = 800):
    x_ss = ((kbp_ss/48.502)*Lc_s_lambda * (coth1((F*2*Lp_s)/kT)-(kT/(F*2*Lp_s))) * (1+(F/S_s)))
    return(x_ss)

In [3]:
def sytox_gap_adjustment(ssDNA_start,ssDNA_end,distance,left_bead_end,right_bead_start,kbp_ss,kbp_ds,pixel_size_um = 0.1):
    kT = 4.11 #Boltzmann constant * temperature in K
    Lp_s = 0.6 # persistence length of ssDNA
    Lc_s_lambda = 27000 # contour length of lambda ssDNA
    S_s = 800 #stretch modulus of ssDNA

    #change parameters to match dsDNA in normal buffer without sytox
    Lp_d = 20.1 #persistence length of dsDNA
    Lc_d_lambda = 16900 #contour length of lambda dsDNA
    S_d = 5399.4 #stretch modulus of dsDNA
    C = 440 #
    g0 = -1.1229e-08 #
    g1 = 18.7 #
    xtotal = distance * 1000

    F = sym.symbols('F')
    eq4 = sym.Eq((((kbp_ss/48.502)*Lc_s_lambda * (coth((F*2*Lp_s)/kT)-(kT/(F*2*Lp_s))) * (1+(F/S_s))) + (kbp_ds/48.502)*Lc_d_lambda * (1 - (0.5 * sqrt(kT/(F*Lp_d))) + (C*F/(((F*g1) + g0)**2 + (S_d*C)))) - xtotal),0)
    pred_force_2 = sym.nsolve(eq4,F,5)
    F = float(pred_force_2)

    ssDNA_len_new = ss_extension(F,kbp_ss)
    ssDNA_len_orig = ssDNA_end - ssDNA_start
    ssDNA_increase = ((ssDNA_len_new/1000)/pixel_size_um) - ssDNA_len_orig
    dsDNA_left_of_gap = ssDNA_start - left_bead_end
    dsDNA_right_of_gap = right_bead_start -ssDNA_end
    dsDNA_left_of_gap_proportion = dsDNA_left_of_gap/(dsDNA_left_of_gap+dsDNA_right_of_gap)
    dsDNA_right_of_gap_proportion = dsDNA_right_of_gap/(dsDNA_left_of_gap+dsDNA_right_of_gap)
    ssDNA_left_adj = dsDNA_left_of_gap_proportion * ssDNA_increase
    ssDNA_right_adj = dsDNA_right_of_gap_proportion * ssDNA_increase
    ssDNA_start = ssDNA_start - ssDNA_left_adj
    ssDNA_end = ssDNA_end + ssDNA_right_adj
    return ssDNA_start,ssDNA_end


In [4]:
def DNA_classification(track_pos_pix,ssDNA_start,ssDNA_end):
    # Create a meshgrid of positions and D_start/D_end
    pos_grid, start_grid = np.meshgrid(track_pos_pix, ssDNA_start)
    _, end_grid = np.meshgrid(track_pos_pix, ssDNA_end)

    # Check conditions
    in_range = (pos_grid >= start_grid) & (pos_grid <= end_grid)
    near_edge = (np.abs(pos_grid - start_grid) <= 2) | (np.abs(pos_grid - end_grid) <= 2)

    # Initialize the result array with 'Neither'
    result_array = np.full(track_pos_pix.shape, 'dsDNA', dtype=object)

    # Update the result array based on conditions
    result_array[np.any(in_range, axis=0)] = 'ssDNA'
    result_array[np.any(near_edge, axis=0)] = 'jDNA'

    return(result_array)

In [5]:
data_folder = '../test_folder/'
kymo_pixelsize_um = 0.1

In [6]:
gap_df_all = pd.DataFrame()

folder_name = data_folder.split('/')[-2]
bead_location_path = f'{data_folder}Bead_locations/'
gap_classification_path = f'{data_folder}sytox_ssDNA_classification/'
gap_fd_fit_path = f'{data_folder}gap_fd_fit/'
refined_tracks_path = f'{data_folder}refined_tracks/long_csv/'
refined_tracks_path_short = f'{data_folder}refined_tracks/short_csv/'
corrected_ssDNA_location_path = f'{data_folder}corrected_ssDNA_location/'
if not os.path.exists(corrected_ssDNA_location_path):
    os.makedirs(corrected_ssDNA_location_path)

bead_location_csvs = os.listdir(bead_location_path)
gap_location_csvs = os.listdir(gap_classification_path)
gap_fd_fit_csvs = os.listdir(gap_fd_fit_path)
refined_tracks = os.listdir(refined_tracks_path)

bead_location_csvs = [f for f in bead_location_csvs if 'kymo' in f]
gap_location_csvs = [f for f in gap_location_csvs if f.endswith('.csv')]
gap_fd_fit_csvs = [f for f in gap_fd_fit_csvs if f.endswith('.csv')]
refined_tracks = [f for f in refined_tracks if f.endswith('.csv')]

markers = list(set([f.split('_')[0] for f in refined_tracks])) 
for marker_name in markers:
    gap_location_for_marker = [f for f in gap_location_csvs if marker_name in f]
    if len(gap_location_for_marker) == 0:continue
    gap_location_for_marker = gap_location_for_marker[0]
    gap_location = pd.read_csv(gap_classification_path+gap_location_for_marker)
    sytox_distance_um = gap_location['Distance'].iloc[0]
    sytox_ssDNA_start = gap_location['ssDNA start'].iloc[0]
    sytox_ssDNA_end = gap_location['ssDNA end'].iloc[0]
    #sytox_gap_size = (sytox_ssDNA_end-sytox_ssDNA_start)*kymo_pixelsize_um

    gap_fd_fit_for_marker = [f for f in gap_fd_fit_csvs if marker_name in f][0]
    gap_fd_fit = pd.read_csv(gap_fd_fit_path+gap_fd_fit_for_marker)
    kbp_ss = gap_fd_fit['kbp_ss'].iloc[0]
    kbp_ds = gap_fd_fit['kbp_ds'].iloc[0]
    

    kymos = [f.split('_')[2].removesuffix('track') for f in refined_tracks if marker_name in f]
    for kymo_id in kymos:
        bead_locations = pd.read_csv(f'{bead_location_path}{marker_name}_kymo_{kymo_id}.csv')
        kymo_force = np.array(bead_locations['Force'])
        left_bead_end_pix = np.array(bead_locations['Left_bead_end'])
        right_bead_start_pix = np.array(bead_locations['Right_bead_start'])
        distance_pix = right_bead_start_pix - left_bead_end_pix
        distance_um = distance_pix * kymo_pixelsize_um
        kymo_duration = bead_locations['Kymo_time'].iloc[0]

        if np.mean(distance_um) < 16:
            ssDNA_start, ssDNA_end = sytox_gap_adjustment(sytox_ssDNA_start,sytox_ssDNA_end,sytox_distance_um,left_bead_end_pix,right_bead_start_pix,kbp_ss,kbp_ds,pixel_size_um = kymo_pixelsize_um)
        else: continue
        corrected_ssDNA_location = pd.DataFrame()
        corrected_ssDNA_location['ssDNA_start'] = ssDNA_start
        corrected_ssDNA_location['ssDNA_end'] = ssDNA_end
        corrected_ssDNA_location.to_csv(f'{corrected_ssDNA_location_path}{marker_name}_kymo{kymo_id}.csv', index=False)

        track_files = [f for f in refined_tracks if f'{marker_name}_kymo_{kymo_id}track_' in f]
        for track_file in track_files:
            track_info = pd.read_csv(f'{refined_tracks_path}{track_file}')
            track_info_short = pd.read_csv(f'{refined_tracks_path_short}{track_file}')
            track_time_pix = np.array(track_info['track_time_pix'])
            track_pos_pix = np.array(track_info['track_pos_pix'])
            ssDNA_start_4_track = ssDNA_start[track_time_pix[0]:track_time_pix[-1]+1]
            ssDNA_end_4_track = ssDNA_end[track_time_pix[0]:track_time_pix[-1]+1]
            DNA_classification_array = DNA_classification(track_pos_pix,ssDNA_start_4_track,ssDNA_end_4_track)
            track_length = len(DNA_classification_array)
            percent_ssDNA = len(DNA_classification_array[DNA_classification_array == 'ssDNA'])/track_length
            percent_dsDNA = len(DNA_classification_array[DNA_classification_array == 'dsDNA'])/track_length
            percent_jDNA = len(DNA_classification_array[DNA_classification_array == 'jDNA'])/track_length
            track_info['DNA_classification'] = DNA_classification_array
            track_info_short['ssDNA_proportion'] = percent_ssDNA
            track_info_short['dsDNA_proportion'] = percent_dsDNA
            track_info_short['jDNA_proportion'] = percent_jDNA
            track_info_short['ssDNA_length'] = np.mean(ssDNA_end) - np.mean(ssDNA_start) - 4
            track_info_short['dsDNA_length'] = np.mean(distance_um)*10 - (np.mean(ssDNA_end) - np.mean(ssDNA_start)) - 4
            track_info_short['jDNA_length'] = 8
            track_info_short['totalDNA_length'] = np.mean(distance_um)*10
            track_info_short['Kymo_length'] = kymo_duration
            track_info_short['Track_start_time'] = track_info['track_time_s'].iloc[0]
            track_info_short['Date'] = folder_name
            track_info_short['Marker'] = marker_name
            track_info_short['Kymo'] = kymo_id
            track_info.to_csv(f'{refined_tracks_path}{track_file}', index = False)
            track_info_short.to_csv(f'{refined_tracks_path_short}{track_file}', index = False)
